In [1]:
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'distrivite.settings.dev')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [31]:
from collections import defaultdict
from decimal import Decimal

from django.db.models import (
    Case,
    DecimalField,
    ExpressionWrapper,
    F,
    DurationField,
    Value,
    When,
)
from django.db.models.functions import Concat

In [32]:
from apps.attendances import models
from django.db.models import Sum        

In [33]:
attendances = models.EmployeeAttendance.objects.all()

In [34]:
start_hours=ExpressionWrapper(
        Case(
        When(check_in_time__lt=F("start_time"), then=F("start_time")),
        default=F("check_in_time"),
    ),

 output_field=DurationField()
)

end_hours=ExpressionWrapper(
        Case(
        When(check_out_time__gt=F("end_time"), then=F("end_time")),
        default=F("check_out_time"),
    ),
 output_field=DurationField()
)

thours=ExpressionWrapper(
        (F("end_time") - F("start_time")), output_field=DurationField()
)

hours=ExpressionWrapper(
        (F("end_hours") - F("start_hours")),
        output_field=DurationField()
)

lhours = ExpressionWrapper(
        F("thours") - F("hours"),
        output_field=DurationField()
)

lremunerations = ExpressionWrapper(
        ((F('lhours__hour')* Value(3600.0) + F('lhours__minute')* Value(60.0) + F('lhours__second')) *Value(0.0002777777777777778)* F('hourly_salary')),
        output_field=DecimalField()
)
remunerations=ExpressionWrapper(
        ((F('hours__hour')* Value(3600.0) + F('hours__minute')* Value(60.0) + F('hours__second')) *Value(0.0002777777777777778)* F('hourly_salary')),
        output_field=DecimalField()
)

In [35]:
phours = ExpressionWrapper(((F("end_time__hour") - F("start_time__hour"))*3600 + (F("end_time__minute") - F("start_time__minute"))*60 + (F("end_time__second") - F("start_time__second"))) /Value(3600) ,output_field=DecimalField()
)

In [36]:
rhours = ExpressionWrapper(((F("check_out_time__hour") - F("check_in_time__hour"))*3600 + (F("check_out_time__minute") - F("check_in_time__minute"))*60 + (F("check_out_time__second") - F("check_in_time__second"))) / Value(3600.0), output_field=DecimalField())

In [37]:
hsalary = ExpressionWrapper(F("hourly_salary"), output_field=DecimalField())

In [38]:
qurery_set = attendances.filter(
    check_in_time__isnull=False,
    check_out_time__isnull=False,
).values("employee_id").annotate(
    phours=phours,
    rhours=rhours,
    hsalary=hsalary,
    psalary = F("phours")*F("hsalary"),
    rsalary = F("rhours")*F("hsalary"),
)
qurery_set

<DataFrameQuerySet [{'employee_id': UUID('9fa2ab21-e922-445b-ab5e-2facbfe4913d'), 'phours': Decimal('10'), 'rhours': Decimal('10'), 'hsalary': Decimal('109.480000000000'), 'psalary': Decimal('1094.80000000000'), 'rsalary': Decimal('1094.80000000000')}, {'employee_id': UUID('9fa2ab21-e922-445b-ab5e-2facbfe4913d'), 'phours': Decimal('10'), 'rhours': Decimal('10'), 'hsalary': Decimal('109.480000000000'), 'psalary': Decimal('1094.80000000000'), 'rsalary': Decimal('1094.80000000000')}, {'employee_id': UUID('fdf6c077-e83b-42f6-b28f-347e9d18ebc3'), 'phours': Decimal('10'), 'rhours': Decimal('0.00416666666666667'), 'hsalary': Decimal('4464.28000000000'), 'psalary': Decimal('44642.8000000000'), 'rsalary': Decimal('18.6011666666667')}, {'employee_id': UUID('85aa7164-eddb-403f-a438-11babdfa8156'), 'phours': Decimal('10'), 'rhours': Decimal('-0.0386111111111111'), 'hsalary': Decimal('2884.61000000000'), 'psalary': Decimal('28846.1000000000'), 'rsalary': Decimal('-111.377997222222')}]>

In [39]:
reports  = qurery_set.values("employee_id", "employee__organization_user__user__last_name").annotate(total_psalary = Sum("psalary"), total_rsalary = Sum("rsalary"), total_phours=Sum("phours"), total_rhours=Sum("rhours")),

In [40]:
reports

(<DataFrameQuerySet [{'employee_id': UUID('85aa7164-eddb-403f-a438-11babdfa8156'), 'employee__organization_user__user__last_name': 'Sob maxime', 'total_psalary': Decimal('28846.1000000000'), 'total_rsalary': Decimal('-111.377997222222'), 'total_phours': Decimal('10'), 'total_rhours': Decimal('-0.0386111111111111')}, {'employee_id': UUID('9fa2ab21-e922-445b-ab5e-2facbfe4913d'), 'employee__organization_user__user__last_name': 'liedji', 'total_psalary': Decimal('2189.60000000000'), 'total_rsalary': Decimal('2189.60000000000'), 'total_phours': Decimal('20'), 'total_rhours': Decimal('20')}, {'employee_id': UUID('fdf6c077-e83b-42f6-b28f-347e9d18ebc3'), 'employee__organization_user__user__last_name': 'moket', 'total_psalary': Decimal('44642.8000000000'), 'total_rsalary': Decimal('18.6011666666667'), 'total_phours': Decimal('10'), 'total_rhours': Decimal('0.00416666666666667')}]>,)

In [42]:
employees = models.Employee.objects.all()

In [45]:
employee_kpi_reports = []
for employee in employees:
    report = [data for data in reports[0] if data["employee_id"] == employee.id]
    if report:
        employee_kpi_reports.append({
            "employee": employee.organization_user.user.last_name,
            "total_phours": report[0]["total_phours"],
            "total_rhours": report[0]["total_rhours"],
            "total_psalary": report[0]["total_psalary"],
            "total_rsalary": report[0]["total_rsalary"],
        })
    else:
        employee_kpi_reports.append({
            "employee": employee.organization_user.user.last_name + " "+ employee.organization_user.user.last_name,
            "total_phours": 0,
            "total_rhours": 0,
            "total_psalary": 0,
            "total_rsalary": 0,
        })
        
print(employee_kpi_reports)

[{'employee': 'liedji', 'total_phours': Decimal('20'), 'total_rhours': Decimal('20'), 'total_psalary': Decimal('2189.60000000000'), 'total_rsalary': Decimal('2189.60000000000')}, {'employee': 'franck franck', 'total_phours': 0, 'total_rhours': 0, 'total_psalary': 0, 'total_rsalary': 0}, {'employee': 'moket', 'total_phours': Decimal('10'), 'total_rhours': Decimal('0.00416666666666667'), 'total_psalary': Decimal('44642.8000000000'), 'total_rsalary': Decimal('18.6011666666667')}, {'employee': 'Sob maxime', 'total_phours': Decimal('10'), 'total_rhours': Decimal('-0.0386111111111111'), 'total_psalary': Decimal('28846.1000000000'), 'total_rsalary': Decimal('-111.377997222222')}]


In [46]:
for employee_kpi in employee_kpi_reports:
    print(employee_kpi)
    print()

{'employee': 'liedji', 'total_phours': Decimal('20'), 'total_rhours': Decimal('20'), 'total_psalary': Decimal('2189.60000000000'), 'total_rsalary': Decimal('2189.60000000000')}

{'employee': 'franck franck', 'total_phours': 0, 'total_rhours': 0, 'total_psalary': 0, 'total_rsalary': 0}

{'employee': 'moket', 'total_phours': Decimal('10'), 'total_rhours': Decimal('0.00416666666666667'), 'total_psalary': Decimal('44642.8000000000'), 'total_rsalary': Decimal('18.6011666666667')}

{'employee': 'Sob maxime', 'total_phours': Decimal('10'), 'total_rhours': Decimal('-0.0386111111111111'), 'total_psalary': Decimal('28846.1000000000'), 'total_rsalary': Decimal('-111.377997222222')}

